# BE 240 Final
Ankita Roychoudhury <br>
6/9/2020

In [1]:
# Basic Imports

#A Model is a CRN with some bells and whistles
from bioscrape.types import Model

#py_simulate_model is a helper function that takes care of may details for you
from bioscrape.simulator import py_simulate_model

#For arrays and plotting
import numpy as np
import pandas as pd
from scipy.integrate import simps
from numpy import trapz

# Import good plotting packages 
import bokeh.io
import bokeh.plotting
from bokeh.layouts import row
from bokeh.layouts import column
bokeh.io.output_notebook()

from biocrnpyler.mechanism import Mechanism
from biocrnpyler.component import Component
from biocrnpyler import Mixture
from biocrnpyler.chemical_reaction_network import Species, Reaction, ComplexSpecies, ChemicalReactionNetwork

import panel as pn
pn.extension()

KeyboardInterrupt: 

In this notebook, I investigate a possible machinery (rheostat) for **ATP regeneration in synthetic cells**. This system, adapted from James Bowie Lab, has shown ATP presence up until 70 hours in buffer. I hope to show ATP life extension with TXTL. In the simulations looking forward, I will try to show that the system with the rheostat will generate more ATP than the case with only ATP leak. The ATP leak parameters will also be present in the system with the rheostat to ensure a true comparison. <br>
Seen below is the entire rheostat machinery:

<div style="width: 500px; margin: auto">

![non-atp](full_rheostat_pic.png)

</div>

This is adopted from a paper in James Bowie Lab (doi: 10.1038/nchembio.2418). 

**The steps taken for modeling are as follows:** <br>
1) Model a Simplified Pathway in Bioscrape <br>
2) Model a Simplified Pathway in BioCRNpyler <br>
3) Model the Entire Pathway in BioCRNpyler <br>

### Simplified Pathway

Now, let's take a look at the simplified model that we begin with: <br>
$$\text{2 ATP + Glucose} \leftrightarrow^{E1} \text{2 ADP + F16P}$$
$$\text{2 ADP + F16P} \leftrightarrow^{E2} \text{2 ATP + Isobutanol}$$
$$\text{3 ADP + Pi + F16P} \leftrightarrow^{E3} \text{3 ATP + Isobutanol}$$
$$\text{ATP} \leftrightarrow^{E4} \text{ADP + Pi}$$ <br> <br>

Essentially, we remove various intermediate reactions. The steps we choose to include are governed by those that require ATP.

## Simplified Pathway in Bioscrape

### Enzymatic Mechanisms
To begin, we will take a look at possible enzymatic mechanisms that may be in play. 

<div style="width: 400px; margin: auto">

![non-atp](enzymatic_models_2.png)

</div>

Here, we are taking a look at some different ways/order in which a fuel, substrate, and enzyme bind to form the waste and product.

Now, we will code up the reactions for the simplified pathway in Bioscrape. We will define three functions, one for each model shown above. Within this function, we will define the species, the reactions (including reverse reactions), the specific rate constants for each constant (governed by a mass action model), and output the dataframe. Note that each function outputs two dataframes, one is the rheostat + atp leak data and the other is atp leak only. The functions will take in parameters, initial conditions for both simulations, and the max time of simulation.

**Model 1**

In [ ]:
def model1_df(parameters, x_0, x_0_atp_leak, t_max):
    '''Return the data for Model 1'''
    
    # 1 step model
    species = ['atp', 'glucose', 'e1', '2atp:glucose:e1', '2adp:f16p:e1', 'adp', 'pi', 'f16p',
           'e2', '2adp:f16p:e2','2atp:isobutanol:e2', 
           '3adppi:f16p:e3', 'e3' , '3atp:isobutanol:e3', 'isobutanol',
          'atp:e4', 'adppi:e4', 'e4']

    # reaction 1
    rxn1_f = (['atp', 'atp', 'glucose', 'e1'], ['2atp:glucose:e1'],
               'massaction',  {'k': 'k_bf'})

    rxn1_r = ( ['2atp:glucose:e1'],['atp', 'atp', 'glucose', 'e1'], 
                'massaction', {'k': 'k_br'})

    # reaction 2
    rxn2_f = ( ['2atp:glucose:e1'],['2adp:f16p:e1'], 
               'massaction', {'k': 'k_cat'})

    # reaction 3
    rxn3_f =  ( ['2adp:f16p:e1'], ['adp', 'adp', 'f16p', 'e1'], 
               'massaction', {'k': 'k_uf'})

    rxn3_r =  (['adp', 'adp', 'f16p', 'e1'], ['2adp:f16p:e1'],
               'massaction', {'k': 'k_ur'})

    # reaction 4
    rxn4_f =  (['f16p', 'adp', 'adp', 'e2'], ['2adp:f16p:e2'],
               'massaction', {'k': 'k_bf'})

    rxn4_r =  ( ['2adp:f16p:e2'], ['f16p', 'adp', 'adp', 'e2'],
               'massaction', {'k': 'k_br'})

    # reaction 5 
    rxn5_f =  (['2adp:f16p:e2'], ['2atp:isobutanol:e2'],
               'massaction', {'k': 'k_cat'})


    # reaction 6
    rxn6_f =  (['2atp:isobutanol:e2'], ['atp', 'atp', 'isobutanol', 'e2'],
               'massaction', {'k': 'k_uf'})

    rxn6_r =  (['atp', 'atp', 'isobutanol', 'e2'], ['2atp:isobutanol:e2'],
               'massaction', {'k': 'k_ur'})


    # reaction 7
    rxn7_f = ( ['adp', 'adp', 'adp', 'pi', 'f16p', 'e3'],['3adppi:f16p:e3'], 
                  'massaction', {'k': 'k_bf'})

    rxn7_r = ( ['3adppi:f16p:e3'],['adp', 'adp', 'adp', 'pi', 'f16p', 'e3'], 
                  'massaction', {'k': 'k_br'})

    # reaction 8
    rxn8_f = ( ['3adppi:f16p:e3'],['3atp:isobutanol:e3'], 
                  'massaction', {'k': 'k_cat'})

    # reaction 9
    rxn9_f = ( ['3atp:isobutanol:e3'],['atp', 'atp','atp',  'isobutanol', 'e3'], 
                  'massaction', {'k': 'k_uf'})

    rxn9_r = ( ['atp', 'atp', 'atp', 'isobutanol', 'e3'],['3atp:isobutanol:e3'], 
                  'massaction', {'k': 'k_ur'})

    # ATP leak reactions
    rxn10_f = (["atp", 'e4'], ['atp:e4'], "massaction", {"k":"k_bf"})
    rxn10_r = (['atp:e4'], ["atp", 'e4'],  "massaction", {"k":"k_br"})

    rxn11_f = (['atp:e4'], ["adppi:e4"],  "massaction", {"k":"k_atp"})

    rxn12_f = ( ["adppi:e4"], ['adp', 'pi', 'e4'], "massaction", {"k":"k_uf"})
    rxn12_r = (['adp', 'pi', 'e4'], ["adppi:e4"] , "massaction", {"k":"k_ur"})

    # Rheostat + ATP leak
    reactions = [rxn1_f, rxn1_r, rxn2_f, rxn3_f, rxn3_r,rxn4_f,
                 rxn4_r, rxn5_f, rxn6_f,  rxn6_r, rxn7_f, rxn8_f, rxn9_f, rxn10_f, rxn10_r,
                rxn11_f, rxn12_f, rxn12_r]

    M = Model(species = species, reactions = reactions, parameters = parameters, initial_condition_dict = x_0)
    timepoints = np.linspace(0,t_max,1000)
    df_results = py_simulate_model(timepoints, Model = M, stochastic = False)

    # ATP Leak only
    species_atp_leak = ['atp', 'adp', 'pi', 'e4', 'atp:e4', 'adppi:e4']
    reactions_atp_leak = [rxn10_f, rxn10_r, rxn11_f, rxn12_f, rxn12_r]
    M_atp_leak = Model(species = species_atp_leak, reactions = reactions_atp_leak, parameters = parameters,
                      initial_condition_dict = x_0_atp_leak)
    df_results_atp_leak = py_simulate_model(timepoints, Model = M_atp_leak, stochastic = False)
    
    return [df_results, df_results_atp_leak]

**Model 2**

In [ ]:
def model2_df(parameters, x_0, x_0_atp_leak, t_max):
    '''Return the data for Model 2'''
    
    # 3 step model v1
    species = ['glucose', 'e1', 'atp', 'glucose:e1', '2atp:glucose:e1', '2adp:f16p:e1', 'adp', 'pi', 'f16p:e1', 'f16p',
              'e2', 'f16p:e2', '2adp:f16p:e2', '2atp:isobutanol:e2', 'isobutanol:e2',
              'e3', 'f16p:e3', '3adppi:f16p:e3', '3atp:isobutanol:e3', 'isobutanol:e3', 'isobutanol',
              'e4', 'atp:e4', 'adppi:e4']


    rxn1_f = (['glucose', 'e1'], ['glucose:e1'], 'massaction', {'k': 'k_bf'})
    rxn1_r = (['glucose:e1'], ['glucose', 'e1'], 'massaction', {'k': 'k_br'})

    rxn2_f = (['atp', 'atp', 'glucose:e1'], ['2atp:glucose:e1'], 'massaction', {'k': 'k_bf'})
    rxn2_r = (['2atp:glucose:e1'], ['atp', 'atp', 'glucose:e1'], 'massaction', {'k': 'k_br'})

    rxn3_f = (['2atp:glucose:e1'], ['2adp:f16p:e1'], 'massaction', {'k': 'k_cat'})

    rxn4_f = (['2adp:f16p:e1'], ['adp', 'adp', 'f16p:e1'], 'massaction', {'k': 'k_uf'})
    rxn4_r = (['adp', 'adp', 'f16p:e1'], ['2adp:f16p:e1'], 'massaction', {'k': 'k_ur'})

    rxn5_f = (['f16p:e1'], ['f16p', 'e1'], 'massaction', {'k': 'k_uf'})
    rxn5_r = (['f16p', 'e1'], ['f16p:e1'], 'massaction', {'k': 'k_ur'})

    rxn6_f = (['f16p', 'e2'], ['f16p:e2'], 'massaction', {'k': 'k_bf'})
    rxn6_r = (['f16p:e2'], ['f16p', 'e2'], 'massaction', {'k': 'k_br'})

    rxn7_f = (['adp', 'adp','f16p:e2'], ['2adp:f16p:e2'], 'massaction', {'k': 'k_bf'})
    rxn7_r = (['2adp:f16p:e2'], ['adp', 'adp', 'f16p:e2'], 'massaction', {'k': 'k_br'})

    rxn8_f = (['2adp:f16p:e2'], ['2atp:isobutanol:e2'], 'massaction', {'k': 'k_cat'})

    rxn9_f = (['2atp:isobutanol:e2'], ['atp', 'atp', 'isobutanol:e2'], 'massaction', {'k': 'k_uf'})
    rxn9_r = (['atp', 'atp', 'isobutanol:e2'], ['2atp:isobutanol:e2'], 'massaction', {'k': 'k_ur'})

    rxn10_f = (['isobutanol:e2'], ['isobutanol', 'e2'], 'massaction', {'k': 'k_uf'})
    rxn10_r = (['isobutanol', 'e2'], ['isobutanol:e2'], 'massaction', {'k': 'k_ur'})

    rxn11_f = (['f16p', 'e3'], ['f16p:e3'], 'massaction', {'k': 'k_bf'})
    rxn11_r = ( ['f16p:e3'], ['f16p', 'e3'], 'massaction', {'k': 'k_br'})

    rxn12_f = (['adp', 'adp', 'adp', 'pi', 'f16p:e3'], ['3adppi:f16p:e3'], 'massaction', {'k': 'k_bf'})
    rxn12_r = (['3adppi:f16p:e3'], ['adp', 'adp', 'adp', 'pi', 'f16p:e3'], 'massaction', {'k': 'k_br'})

    rxn13_f = (['3adppi:f16p:e3'], ['3atp:isobutanol:e3'], 'massaction', {'k': 'k_cat'})

    rxn14_f = (['3atp:isobutanol:e3'], ['atp', 'atp', 'atp', 'isobutanol:e3'], 'massaction', {'k': 'k_uf'})
    rxn14_r = ( ['atp', 'atp', 'atp', 'isobutanol:e3'], ['3atp:isobutanol:e3'], 'massaction', {'k': 'k_ur'})

    rxn15_f = (['isobutanol:e3'], ['isobutanol', 'e3'], 'massaction', {'k': 'k_uf'})
    rxn15_r = (['isobutanol', 'e3'], ['isobutanol:e3'], 'massaction', {'k': 'k_ur'})

    #ATP Leak reactions
    rxn16_f = (["atp", 'e4'], ['atp:e4'], "massaction", {"k":"k_bf"})
    rxn16_r = (['atp:e4'], ["atp", 'e4'],  "massaction", {"k":"k_br"})

    rxn17_f = (['atp:e4'], ["adppi:e4"],  "massaction", {"k":"k_atp"})

    rxn18_f = ( ["adppi:e4"], ['adp', 'pi', 'e4'], "massaction", {"k":"k_uf"})
    rxn18_r = (['adp', 'pi', 'e4'], ["adppi:e4"] , "massaction", {"k":"k_ur"})

    # Rheostat + ATP Leak
    reactions = [rxn1_f, rxn1_r, rxn2_f, rxn2_r, rxn3_f, rxn4_f, rxn4_r, rxn5_f, rxn5_r, rxn6_f, rxn6_r, rxn7_f, rxn7_r,
                rxn8_f, rxn9_f, rxn9_r, rxn10_f, rxn10_r, rxn11_f, rxn11_r, rxn12_f, rxn12_r, rxn13_f,
                rxn14_f, rxn14_r, rxn15_f, rxn15_r, 
                rxn16_f, rxn16_r, rxn17_f, rxn18_f, rxn18_r]

    M = Model(species = species, reactions = reactions, parameters = parameters,
             initial_condition_dict = x_0)
    timepoints = np.linspace(0,t_max,1000)
    df_results = py_simulate_model(timepoints, Model = M, stochastic = False)

    # ATP Leak only
    species_atp_leak = ['atp', 'adp', 'pi', 'e4', 'atp:e4', 'adppi:e4']
    reactions_atp_leak = [rxn16_f, rxn16_r, rxn17_f, rxn18_f, rxn18_r]
    M_atp_leak = Model(species = species_atp_leak, reactions = reactions_atp_leak, parameters = parameters,
             initial_condition_dict = x_0_atp_leak)
    df_results_atp_leak = py_simulate_model(timepoints, Model = M_atp_leak, stochastic = False)
    
    return [df_results, df_results_atp_leak]

**Model 3**

In [ ]:
def model3_df(parameters, x_0, x_0_atp_leak, t_max):
    
    '''Output data for Model 3'''
    
    species = ['e1', 'atp', 'e1:2atp', 'glucose', '2atp:glucose:e1', '2adp:f16p:e1', '2adp:e1', 'f16p',
          '2adp:e1', 'adp',
          'e2', 'e2:2adp', '2adp:f16p:e2', '2atp:isobutanol:e2', '2atp:e2', 'isobutanol',
          'e3', 'pi', 'e3:3adppi', '3adppi:f16p:e3', '3atp:f16p:e3', '3atp:e3', 'e3', 'e4', 'atp:e4', 'adppi:e4']

    rxn1_f = (['e1', 'atp', 'atp'], ['e1:2atp'], 'massaction', {'k': 'k_bf'})
    rxn1_r = (['e1:2atp'], ['e1', 'atp', 'atp'], 'massaction', {'k': 'k_br'})

    rxn2_f = (['glucose', 'e1:2atp'], ['2atp:glucose:e1'], 'massaction', {'k': 'k_bf'})
    rxn2_r = (['2atp:glucose:e1'], ['glucose', 'e1:2atp'], 'massaction', {'k': 'k_br'})

    rxn3_f = (['2atp:glucose:e1'], ['2adp:f16p:e1'], 'massaction', {'k': 'k_cat'})

    rxn4_f = (['2adp:f16p:e1'], ['2adp:e1', 'f16p'], 'massaction', {'k': 'k_uf'})
    rxn4_r = (['2adp:e1', 'f16p'], ['2adp:f16p:e1'], 'massaction', {'k': 'k_ur'})

    rxn5_f = (['2adp:e1'], ['adp', 'adp', 'e1'], 'massaction', {'k': 'k_uf'})
    rxn5_r = (['adp', 'adp', 'e1'], ['2adp:e1'], 'massaction', {'k': 'k_ur'})

    rxn6_f = (['adp', 'adp','e2'], ['e2:2adp'], 'massaction', {'k': 'k_bf'})
    rxn6_r = (['e2:2adp'], ['adp', 'adp','e2'], 'massaction', {'k': 'k_br'})

    rxn7_f = (['f16p', 'e2:2adp'], ['2adp:f16p:e2'], 'massaction', {'k': 'k_bf'})
    rxn7_r = (['2adp:f16p:e2'], ['f16p', 'e2:2adp'], 'massaction', {'k': 'k_br'})

    rxn8_f = (['2adp:f16p:e2'], ['2atp:isobutanol:e2'], 'massaction', {'k': 'k_cat'})

    rxn9_f = (['2atp:isobutanol:e2'], ['2atp:e2', 'isobutanol'], 'massaction', {'k': 'k_uf'})
    rxn9_r = (['2atp:e2', 'isobutanol'], ['2atp:isobutanol:e2'], 'massaction', {'k': 'k_ur'})

    rxn10_f = (['2atp:e2'], ['atp', 'atp', 'e2'], 'massaction', {'k': 'k_uf'})
    rxn10_r = (['atp', 'atp', 'e2'], ['2atp:e2'], 'massaction', {'k': 'k_ur'})

    rxn11_f = (['e3', 'adp', 'adp', 'adp', 'pi'], ['e3:3adppi'], 'massaction', {'k': 'k_bf'})
    rxn11_r = (['e3:3adppi'], ['e3', 'adp', 'adp', 'adp', 'pi'], 'massaction', {'k': 'k_br'})

    rxn12_f = (['e3:3adppi', 'f16p'], ['3adppi:f16p:e3'], 'massaction', {'k': 'k_bf'})
    rxn12_r = (['3adppi:f16p:e3'], ['e3:3adppi', 'f16p'], 'massaction', {'k': 'k_br'})

    rxn13_f = (['3adppi:f16p:e3'], ['3atp:f16p:e3'], 'massaction', {'k': 'k_cat'})

    rxn14_f = (['3atp:f16p:e3'], ['3atp:e3', 'f16p'], 'massaction', {'k': 'k_uf'})
    rxn14_r = (['3atp:e3', 'f16p'], ['3atp:f16p:e3'], 'massaction', {'k': 'k_ur'})

    rxn15_f = (['3atp:e3'], ['atp', 'atp', 'atp', 'e3'], 'massaction', {'k': 'k_uf'})
    rxn15_r = (['atp', 'atp', 'atp', 'e3'], ['3atp:e3'], 'massaction', {'k': 'k_ur'})

    # ATP Leak reactions
    rxn16_f = (["atp", 'e4'], ['atp:e4'], "massaction", {"k":"k_bf"})
    rxn16_r = (['atp:e4'], ["atp", 'e4'],  "massaction", {"k":"k_br"})

    rxn17_f = (['atp:e4'], ["adppi:e4"],  "massaction", {"k":"k_atp"})

    rxn18_f = ( ["adppi:e4"], ['adp', 'pi', 'e4'], "massaction", {"k":"k_uf"})
    rxn18_r = (['adp', 'pi', 'e4'], ["adppi:e4"] , "massaction", {"k":"k_ur"})

    # Rheostat and ATP Leak
    reactions = [rxn1_f, rxn1_r, rxn2_f, rxn2_r, rxn3_f, rxn4_f, rxn4_r,
            rxn5_f, rxn5_r, rxn6_f, rxn6_r, rxn7_f, rxn7_r, rxn8_f,
            rxn9_f, rxn9_r, rxn10_f, rxn10_r, rxn11_f, rxn11_r, rxn12_f,
            rxn12_r, rxn13_f, rxn14_f, rxn14_r, rxn15_f,rxn15_r,
             rxn16_f, rxn16_r, rxn17_f, rxn18_f, rxn18_r]

    M = Model(species = species, reactions = reactions, parameters = parameters,
             initial_condition_dict = x_0)
    timepoints = np.linspace(0,t_max,1000)
    df_results = py_simulate_model(timepoints, Model = M, stochastic = False)

    # ATP Leak only
    species_atp_leak = ['atp', 'adp', 'pi', 'e4', 'atp:e4', 'adppi:e4']
    reactions_atp_leak = [rxn16_f, rxn16_r, rxn17_f, rxn18_f, rxn18_r]
    M_atp_leak = Model(species = species_atp_leak, reactions = reactions_atp_leak, parameters = parameters,
             initial_condition_dict = x_0_atp_leak)
    df_results_atp_leak = py_simulate_model(timepoints, Model = M_atp_leak, stochastic = False)

    return [df_results, df_results_atp_leak]

Now, we will define a function that plots our data. It will take in the parameters for each set of reactions, the initial conditions, and t_max.

In [ ]:
def plot_all(parameters1, parameters2, parameters3, x_0, x_0_atp_leak, t_max = 100):

    # Define time points and color list
    timepoints = np.linspace(0,t_max,1000)
    colors=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02','#a6761d']
    
    # find data for MODEL ONE
    model_1_df, model_1_df_atp_leak = model1_df(parameters1, x_0, x_0_atp_leak, t_max)
    
    # Plot glucose and isobutanol for model 1
    p1 = bokeh.plotting.figure(width = 400, height = 225, 
                             x_axis_label = 'time (hrs)',
                             y_axis_label = 'concentration (mM)',
                             title = '1 Step Model (massaction)')
    p1.line(timepoints, model_1_df['glucose'], color = colors[0],line_width = 2, legend_label = 'glucose')
    p1.line(timepoints, model_1_df['isobutanol'], color = colors[1], line_width = 2,legend_label = 'isobutanol')
    p1.line(timepoints, model_1_df['f16p'], color = colors[2], line_width = 2,legend_label = 'f16p')
    p1.legend.location = 'center_right'

    # Plot ATP and ADP, Pi for model 1
    p2 = bokeh.plotting.figure(width = 400, height = 225,
                               x_axis_label = 'time (hrs)',
                             y_axis_label = 'concentration (mM)',
                             title = 'ATP Compare')#  f" {parameters1}")
    p2.line(timepoints, model_1_df['atp'], color = colors[3], line_width = 2,legend_label = 'atp')
    p2.line(timepoints, model_1_df_atp_leak['atp'], legend_label = 'atp only',line_width=2, color = colors[4])

    p2.line(timepoints, model_1_df['pi'], color = 'silver',line_width = 2, legend_label = 'pi') #colors[6]
    p2.line(timepoints, model_1_df['adp'], color = 'silver', line_width = 2,legend_label = 'adp') #colors[5]
    p2.legend.location = 'center_right'


    # find data for MODEL TWO
    model_2_df, model_2_df_atp_leak = model2_df(parameters2, x_0, x_0_atp_leak, t_max)
    
    # Plot glucose and isobutanol for model 2
    p4 = bokeh.plotting.figure(width = 400, height = 225, 
                             x_axis_label = 'time (hrs)',
                             y_axis_label = 'concentration (mM)',
                             title = '3 Step Model V1 (massaction)')
    p4.line(timepoints, model_2_df['glucose'], color = colors[0],line_width = 2,  legend_label = 'glucose')
    p4.line(timepoints, model_2_df['isobutanol'], color = colors[1], line_width = 2, legend_label = 'isobutanol')
    p4.line(timepoints, model_2_df['f16p'], color = colors[2], line_width = 2, legend_label = 'f16p')
    p4.legend.location = 'center_right'

    # Plot ATP and ADP, Pi for model 2
    p5 = bokeh.plotting.figure(width = 400, height = 225,
                               x_axis_label = 'time (hrs)',
                             y_axis_label = 'concentration (mM)',
                              title =  'ATP Compare')
    p5.line(timepoints, model_2_df['atp'], color = colors[3],line_width = 2,  legend_label = 'atp')
    p5.line(timepoints, model_2_df_atp_leak['atp'], legend_label = 'atp only', line_width = 2, color = colors[4])

    p5.line(timepoints, model_2_df['pi'], color = 'silver',line_width = 2, legend_label = 'pi') #colors[6], 
    p5.line(timepoints, model_2_df['adp'], color = 'silver',line_width = 2, legend_label = 'adp')#colors[5], 
    p5.legend.location = 'center_right'
    
    
    # find data for MODEL THREE
    model_3_df, model_3_df_atp_leak = model3_df(parameters3, x_0, x_0_atp_leak, t_max)
 
      # Plot glucose and isobutanol for model 3
    p7 = bokeh.plotting.figure(width = 400, height = 225, 
                             x_axis_label = 'time (hrs)',
                             y_axis_label = 'concentration (mM)',
                             title = '3 Step Model V2 (massaction)')
    p7.line(timepoints, model_3_df['glucose'], color = colors[0], line_width = 2, legend_label = 'glucose')
    p7.line(timepoints, model_3_df['isobutanol'], color = colors[1],line_width = 2,  legend_label = 'isobutanol')
    p7.line(timepoints, model_3_df['f16p'], color = colors[2],line_width = 2,  legend_label = 'f16p')
    p7.legend.location = 'center_right'

    # Plot ATP and ADP, Pi for model 3
    p8 = bokeh.plotting.figure(width = 400, height = 225,
                               x_axis_label = 'time (hrs)',
                             y_axis_label = 'concentration (mM)',
                              title =  "ATP Compare")
    p8.line(timepoints, model_3_df['atp'], color = colors[3],line_width = 2,  legend_label = 'atp')
    p8.line(timepoints, model_3_df_atp_leak['atp'], line_width = 2, legend_label = 'atp only', color = colors[4])

    p8.line(timepoints, model_3_df['pi'], color = 'silver',line_width = 2,  legend_label = 'pi')#colors[6]
    p8.line(timepoints, model_3_df['adp'], color = 'silver', line_width = 2, legend_label = 'adp')#colors[5]
    p8.legend.location = 'center_right'

    

    bokeh.io.show(row(p1, p2))
    bokeh.io.show(row(p4, p5))
    bokeh.io.show(row(p7, p8))
    
    # Return for future comparison with biocrnpyler
    return p1, p2

Now, let's define our parameters and initial conditions. All concentrations are in units mM and rate constants in units 1/hr.

In [ ]:
# Define initial conditions for Rheostat + ATP Leak
e = 0.15 
e4 = 3.5
atp = 30
x_0 = {
    "glucose":atp, 
    "atp":atp, 
    'e1':e, 
    'e2':e,
    'e3':e,
    'e4':e4,
}

# Initial conditions for ATP Leak only
x_0_atp_leak = { 
    'atp':atp,
    'e4':e4
}

# Parameters for all 3 models
parameters1 = [('k_bf', 20), ('k_br', 2.0), ('k_uf', 20), ('k_ur', 2.0), ('k_cat', 10),('k_atp', 1)]
parameters2 = [('k_bf', 20), ('k_br', 2.0), ('k_uf', 20), ('k_ur', 2.0), ('k_cat', 10),('k_atp', 1)]
parameters3 = [('k_bf', 20), ('k_br', 2.0), ('k_uf', 20), ('k_ur', 2.0), ('k_cat', 10),('k_atp', 1)]

Now, let's take a look at the plots!

In [ ]:
p1_bioscrape, p2_bioscrape = plot_all(parameters1, parameters2, parameters3, x_0, x_0_atp_leak, 72)

As we can see here, we see fairly similar dynamics for all three models. The left column of plots show the use of glucose, the production of isobutanol, and the presence of an intermediate, f16p. The plots on the right show the ATP comparison. The curves for ATP, Pi, and ADP come from the Rheostat + ATP Leak model. The curves for atp only comes from the ATP Leak Only model. Preferably, we would like the atp line to last longer and be right shifted compared to the atp only model. We see these dynamics best in the first model, so we decide to move forward with it. It is also the simplest model.

## Simplified Pathway in BioCRNpyler

Now, we will define a new function, Mechanism, Component, and Mixture to account for the desired energy use dynamics. The function (check_type) will check what type of item the species is and return the desired type. BioCRNpyler's self.set_species has the same functionality, however it does not work for mechanisms since they do not have a set_species function. In those cases, we will use our check_type function. The Mechanisms will be called **FuelMichaelisMenten**, it will be of type 'catalysis,' it will take in an enzyme name and a _list_ of fuels, substrates, products, and wastes that are involved with that enzyme. It will consist of a binding, catalysis, and unbinding reaction, as shown in Model 1 above. We will create a Component called **Enzyme**. It will use mechanism 'catalysis' and will take in particular binding rates($ k_{uf}, k_{bf}, k_{cat}$). We will also define a new Mixture called **EnergyTxTl** that will use mechanism 'catalysis.'<br><br>
### check_type Function

In [ ]:
def check_type(item, material_type_str): 
    if isinstance(item, Species):
        item_ret = item
    elif isinstance(item, str):
        item_ret = Species(name = item, material_type = material_type_str)
    elif isinstance(item, Component) and item.get_species() != None:
        item_ret = item.get_species()
    else:
        raise ValueError( item, "parameter must be a string, a Component with defined get_species(), or a chemical_reaction_network.species")
        
    return item_ret

### Mechanism: FuelMichaelisMenten

In [ ]:
class FuelMichaelisMenten(Mechanism):
    def __init__(self, name, type = 'catalysis', **keywords):
        
        Mechanism.__init__(self = self, name = name, mechanism_type = type, **keywords)

    def update_species(self, enzyme, fuel_list, substrate_list, product_list, waste_list): 
        
        self.enzyme = check_type(enzyme, 'enzyme')
        
        species = [self.enzyme]
        comp1_list = [self.enzyme]
        comp2_list = [self.enzyme]
        
        for f in fuel_list:
            species.append(f)
            comp1_list.append(f)

        for s in substrate_list:
            species.append(s)
            comp1_list.append(s)
            
        for p in product_list:
            species.append(p)
            comp2_list.append(p)
            
        for w in waste_list:
            species.append(w)
            comp2_list.append(w)
               
        
        species += [ComplexSpecies(comp1_list)]
        species += [ComplexSpecies(comp2_list)]
        return species
    
    def update_reactions(self, enzyme, fuel_list, substrate_list, product_list, waste_list, k_bf, k_uf, k_cat, component = None,
                        part_id = None): 
        
        # Reverse binding rates
        k_br = 0.1*k_bf
        k_ur = 0.1*k_uf

        self.enzyme = check_type(enzyme, 'enzyme')
        
        # Define input lists
        comp1_list = [self.enzyme]
        comp2_list = [self.enzyme]
        
        for f in fuel_list:
            comp1_list.append(f)
            
        for s in substrate_list:
            comp1_list.append(s)

        for p in product_list:
            comp2_list.append(p)
            
        for w in waste_list:
            comp2_list.append(w)
        
        # Define Complexes
        comp1 = ComplexSpecies(comp1_list)
        comp2 = ComplexSpecies(comp2_list)
        
        # Define Reactions
        binding_rxn = Reaction(inputs = comp1_list, outputs=[comp1], k = k_bf, k_rev = k_br)
        cat_rxn = Reaction(inputs = [comp1], outputs = [comp2], k = k_cat)
        unbinding_rxn = Reaction(inputs = [comp2], outputs = comp2_list, k=k_uf, k_rev = k_ur)
        
        return [binding_rxn, cat_rxn, unbinding_rxn]

### Component: Enzyme

In [ ]:
class Enzyme(Component):
    def __init__(self, enzyme_name, substrate, fuel, product, waste, k_bf, k_uf, k_cat = 36000, **keywords):
      
        # ENZYME NAME
        self.enzyme = check_type(enzyme_name, 'enzyme')
    
        # SUBSTRATE
        self.substrate_list = []
        for s in substrate:
            self.substrate_list.append(self.set_species(s, material_type = 'molecule'))
            
        # FUEL
        self.fuel_list = []
        for f in fuel:
            self.fuel_list.append(self.set_species(f, material_type ='metabolite'))
        
        
        # PRODUCT
        self.product_list = []
        for p in product:
            self.product_list.append(self.set_species(p, material_type = 'molecule'))
          
            
        # WASTE
        self.waste_list = []
        for w in waste:
            self.waste_list.append(self.set_species(w, material_type = 'metabolite'))
        
        
        self.k_bf = k_bf
        self.k_uf = k_uf
        self.k_cat = k_cat
      
        Component.__init__(self = self, name = enzyme_name, **keywords)
        
    def update_species(self):
        mech_cat = self.mechanisms['catalysis']

        return mech_cat.update_species(self.enzyme, self.fuel_list, self.substrate_list, self.product_list, self.waste_list) 
                                                                                           
    
    def update_reactions(self):
        mech_cat = self.mechanisms['catalysis']

        return mech_cat.update_reactions(self.enzyme, self.fuel_list, self.substrate_list, self.product_list, self.waste_list,self.k_bf,
                                         self.k_uf, self.k_cat,component = None,  part_id = None)
    

### Mixture: EnergyTxTl

In [ ]:
class EnergyTxTl(Mixture):
    def __init__(self, name="",**keywords): 
        

        mech_cat = FuelMichaelisMenten('catalysis')
        
        default_mechanisms = {
            mech_cat.mechanism_type:mech_cat
        }
        
        Mixture.__init__(self, name = name, default_mechanisms=default_mechanisms, **keywords)     

### Simulate Simplified BioCRNpyler

Now, we will perform the simulation. To do this, we begin by defining rate constants and the appropriate input arguments of each Enzyme. Note that we will use the same initial conditions, rate constants, and reactions as the above simplified simulation in Bioscrape.

In [ ]:
# Define rate constants
k_bf = 20
k_uf = 20

# Define reaction for each enzyme
E1 = Enzyme(enzyme_name = "enzyme1", substrate = ['glucose'],
            fuel = ['atp', 'atp'],product = ['f16p'], 
            waste = ['adp','adp'], k_bf = k_bf, k_uf = k_uf, k_cat = 10)

E2 = Enzyme(enzyme_name = 'enzyme2', substrate = ['f16p'], 
            fuel = ['adp', 'adp'],
           product = ['isobutanol'], waste = ['atp', 'atp'],
           k_bf = k_bf, k_uf = k_uf, k_cat = 10)

E3 = Enzyme(enzyme_name = 'enzyme3', substrate = ['f16p'], 
            fuel = ['adp', 'adp', 'adp', 'pi'], product = ['isobutanol'],
           waste = ['atp', 'atp', 'atp']
           , k_bf = k_bf, k_uf = k_uf, k_cat = 10)

E4 = Enzyme(enzyme_name = 'enzyme4', substrate = [], 
            fuel = ['atp'], product = [], waste = ['adp', 'pi'],
             k_bf = k_bf, k_uf = k_uf,
            k_cat = 1)

# Define the Mixture
myMixture = EnergyTxTl(components = [E1,E2,E3,E4])
CRN = myMixture.compile_crn()

myMixture_atp = EnergyTxTl(components = [E4])
CRN_atp = myMixture_atp.compile_crn()

# print(CRN.pretty_print(show_rates = False, show_attributes = False, show_materials = False))
# print('\n ATP', CRN_atp.pretty_print(show_rates = False))

Now, we will define timepoints, initial conditions and run the simulation.

In [ ]:
# CRN.write_sbml_file("CRN.sbml")
# CRN_atp.write_sbml_file("CRN_atp.sbml")

# Define timepoints
timepoints = np.linspace(0,72,100)

# Define initial conditions for Rheostat + ATP Leak
e = 0.15
e4 = 3.5
atp = 30
x0 = {'molecule_glucose':30,
      'metabolite_atp': atp,
      "enzyme_enzyme1":e,
      "enzyme_enzyme2":e,
      "enzyme_enzyme3":e,
      "enzyme_enzyme4":e4
}

# Define initial conditions for ATP Leak only
x0_atp = {'metabolite_atp':atp,
          "enzyme_enzyme4":e4,

    }

# Perform simulations
re = CRN.simulate_with_bioscrape(timepoints, initial_condition_dict = x0)
re_atp = CRN_atp.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_atp)

Now, we will take a look at the plots!

In [ ]:
colors=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02','#a6761d']
p1_biocrn = bokeh.plotting.figure(width = 400, height = 225, 
                             x_axis_label = 'time',
                             y_axis_label = 'concentration (mM)',
                             title = 'Biocrnpyler Simple')
p1_biocrn.line(timepoints, re['molecule_glucose'], color = colors[0],line_width = 2, legend_label = 'glucose')
p1_biocrn.line(timepoints, re['molecule_isobutanol'], color = colors[1], line_width = 2,legend_label = 'isobutanol')
p1_biocrn.line(timepoints, re['molecule_f16p'], color = colors[2], line_width = 2,legend_label = 'f16p')
p1_biocrn.legend.location = 'center_right'

# Plot ATP and ADP, Pi
p2_biocrn = bokeh.plotting.figure(width = 400, height = 225,
                           x_axis_label = 'time (hrs)',
                         y_axis_label = 'concentration',
                         title = 'ATP Compare')#  f" {parameters1}")
p2_biocrn.line(timepoints, re['metabolite_atp'], color = colors[3], line_width = 2,legend_label = 'atp')
p2_biocrn.line(timepoints, re_atp['metabolite_atp'], legend_label = 'atp only',line_width=2, color = colors[4])

p2_biocrn.line(timepoints, re['metabolite_pi'], color = 'silver',line_width = 2, legend_label = 'pi') #colors[6]
p2_biocrn.line(timepoints, re['metabolite_adp'], color = 'silver', line_width = 2,legend_label = 'adp') #colors[5]
p2_biocrn.legend.location = 'center_right'

p1_bioscrape.title.text = 'Bioscrape Simple'
bokeh.io.show(row(p1_bioscrape, p2_bioscrape))
bokeh.io.show(row(p1_biocrn, p2_biocrn))

On the first row, we have plotted the simulation data for the simplified system from the Bioscrape model. On the second row, we have plotted the simulation data for the simplified system from the BioCRNpyler model. We can see that they are quite similar! To be sure, let's take a look at an overlay of the two simulations.

In [ ]:
p1_overlay = p1_bioscrape
p2_overlay = p2_bioscrape

p1_overlay.line(timepoints, re['molecule_glucose'], color = colors[0],line_width = 2, legend_label = 'glucose')
p1_overlay.line(timepoints, re['molecule_isobutanol'], color = colors[1], line_width = 2,legend_label = 'isobutanol')
p1_overlay.line(timepoints, re['molecule_f16p'], color = colors[2], line_width = 2,legend_label = 'f16p')
p1_overlay.title.text = 'Overlay'
p1_overlay.legend.location = 'center_right'


p2_overlay.line(timepoints, re['metabolite_atp'], color = colors[3], line_width = 2,legend_label = 'atp')
p2_overlay.line(timepoints, re_atp['metabolite_atp'], legend_label = 'atp only',line_width=2, color = colors[4])

p2_overlay.line(timepoints, re['metabolite_pi'], color = 'silver',line_width = 2, legend_label = 'pi') #colors[6]
p2_overlay.line(timepoints, re['metabolite_adp'], color = 'silver', line_width = 2,legend_label = 'adp') #colors[5]
p2_overlay.legend.location = 'center_right'
p2_overlay.title.text = ''

bokeh.io.show(row(p1_overlay, p2_overlay))

I found that this similarity was sufficient to move forward. Now, we will use the BioCRNpyler Component, Mechanism, and Mixture to simulate the entire rheostat pathway!

## Entire Pathway in BioCRNpyler

In [ ]:
# Define rate constants
k_bf = 20
k_uf = 20 

# Define reaction for each enzyme
E1_hex = Enzyme(enzyme_name = "hex", substrate = ['glucose'],
            fuel = ['atp'],product = ['g6p'], waste = ['adp'], k_bf = k_bf , k_uf = k_uf)

E2_pgi = Enzyme(enzyme_name = 'pgi', substrate = ['g6p'], fuel = [],
           product = ['f6p'], waste = [], k_bf = k_bf , k_uf = k_uf)

E3_pfk = Enzyme(enzyme_name = 'pfk', substrate = ['f6p'], fuel = ['atp'], product = ['f16p'],
           waste = ['adp'], k_bf = k_bf , k_uf = k_uf)

E4_ald_tpi = Enzyme(enzyme_name ='ald_tpi' , substrate = ['f16p'], fuel = [], product = ['g3p', 'g3p'], 
            waste = [], k_bf = k_bf , k_uf = k_uf )

E5_gapN = Enzyme(enzyme_name ='gapN' , substrate = ['g3p', 'g3p'], fuel = ['nadp', 'nadp'], product = ['3pg', '3pg'], 
            waste = ['nadph', 'nadph'], k_bf = k_bf , k_uf = k_uf)

E6_gapM6 = Enzyme(enzyme_name ='gapM6' , substrate = ['g3p', 'g3p'], fuel = ['pi'], product = ['13bpg'], 
            waste = [],k_bf = k_bf , k_uf = k_uf)

E7_pgk = Enzyme(enzyme_name = 'pgk', substrate = ['13bpg'], fuel = ['adp'], product = ['3pg', '3pg'], 
            waste = ['atp'], k_bf = k_bf , k_uf = k_uf)

E8_pgm = Enzyme(enzyme_name ='pgm' , substrate = ['3pg', '3pg'], fuel = [], product = ['2pg', '2pg'], 
            waste = [], k_bf = k_bf , k_uf = k_uf)

E9_eno = Enzyme(enzyme_name ='eno' , substrate = ['2pg', '2pg'], fuel = [], product = ['pep', 'pep'], 
            waste = [],k_bf = k_bf , k_uf = k_uf)

E10_pyk = Enzyme(enzyme_name = 'pyk', substrate = ['pep', 'pep'], fuel = ['adp', 'adp'], product = ['pyruvate', 'pyruvate'], 
            waste = ['atp', 'atp'], k_bf = k_bf , k_uf = k_uf) # irreversible

E11_alsS = Enzyme(enzyme_name = 'alsS', substrate = ['pyruvate', 'pyruvate'], fuel = [], product = ['acetolac'], 
            waste = [], k_bf = k_bf , k_uf = k_uf) # irreversible

E12_IlvC = Enzyme(enzyme_name = 'IlvC', substrate = ['acetolac'], fuel = ['nadph'], product = ['23dih3mebut'], 
            waste = ['nadp'], k_bf = k_bf , k_uf = k_uf)

E13_IlvD = Enzyme(enzyme_name ='IlvD' , substrate = ['23dih3mebut'], fuel = [], product = ['3me2oxo'], 
            waste = [],k_bf = k_bf , k_uf = k_uf)

E14_kivD = Enzyme(enzyme_name ='kivD' , substrate = ['3me2oxo'], fuel = [], product = ['isobutanal'], 
            waste = [], k_bf = k_bf , k_uf = k_uf) # irreversible

E15_yahk = Enzyme(enzyme_name = 'yahk', substrate = ['isobutanal'], fuel = ['nadph'], product = ['isobutanol'],
                  waste = ['nadp'], k_bf = k_bf , k_uf = k_uf)

# Define ATP Leak Only Enzyme
E16_all_other_atp = Enzyme(enzyme_name = 'atp_synthase', substrate = [], fuel = ['atp'], 
                           product = [], waste = ['adp', 'pi'], k_bf = k_bf, k_uf = k_uf, k_cat = 10,)

# Define the Mixture for Rheostat + ATP Leak
myMixture = EnergyTxTl(components = [E1_hex,E2_pgi,E3_pfk, E4_ald_tpi, E5_gapN, E6_gapM6, E7_pgk, E8_pgm, E9_eno, E10_pyk, 
                                    E11_alsS, E12_IlvC, E13_IlvD, E14_kivD, E15_yahk, E16_all_other_atp])
CRN = myMixture.compile_crn()

# Define Mixture for ATP Leak only
myMixture_atp = EnergyTxTl(components = [E16_all_other_atp])
CRN_atp = myMixture_atp.compile_crn()


Now, we will define timepoints, initial conditions, and perform the simulation.

In [ ]:
# CRN.write_sbml_file("CRN.sbml")
# CRN_atp.write_sbml_file("CRN_atp.sbml")

# Define timepoints
timepoints = np.linspace(0,72,100)

# Define initial conditions for Rheostat + ATP Leak Model
e = 0.15
e4 = 3.5
atp = 30
x0 = {'molecule_glucose':30,
      'metabolite_atp': atp,
     'metabolite_nadp':atp,
      "enzyme_hex":e,
      'enzyme_pgi':e,
      'enzyme_pfk':e,
      'enzyme_ald_tpi':e,
      'enzyme_gapN':e, 
      'enzyme_gapM6':e,
      'enzyme_pgk':e,
      'enzyme_pgm':e,
      'enzyme_eno':e,
      'enzyme_pyk':e,
      'enzyme_alsS':e,
      'enzyme_IlvC':e,
      'enzyme_IlvD':e,
      'enzyme_kivD':e,
      'enzyme_yahk':e,
      "enzyme_atp_synthase":e4
}
     
# Define initial conditions for ATP Leak Model Only
x0_atp = {'metabolite_atp':atp,
          "enzyme_atp_synthase":e4,
}

# Perform simulations
re = CRN.simulate_with_bioscrape(timepoints, initial_condition_dict = x0)
re_atp = CRN_atp.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_atp)

Now, let's take a look at the plots!

In [ ]:
# same color scheme as the others for presentation
colors=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02','#a6761d']
#colors = ['#8dd3c7','#ffffb3','#bebada','#fb8072','#80b1d3','#fdb462','#b3de69','#fccde5','#d9d9d9','#bc80bd','#ccebc5','#ffed6f']

# First plot glucose and isobutanol
p1_complex = bokeh.plotting.figure(width = 450, height = 250, 
                         x_axis_label = 'time',
                         y_axis_label = 'concentration (mM)',
                         title = 'Entire Pathway Biocrnpyler with previously used parameters')
p1_complex.line(timepoints, re['molecule_glucose'], color = colors[0],line_width = 2, legend_label = 'glucose')
p1_complex.line(timepoints, re['molecule_isobutanol'], color = colors[1], line_width = 2,legend_label = 'isobutanol')
p1_complex.line(timepoints, re['molecule_f16p'], color = colors[2], line_width = 2,legend_label = 'f16p')
p1_complex.legend.location = 'center_right'
p1_complex.legend.click_policy="hide"


# Plot ATP and ADP, Pi
p2_complex = bokeh.plotting.figure(width = 450, height = 250,
                           x_axis_label = 'time (hrs)',
                         y_axis_label = 'concentration')
p2_complex.line(timepoints, re['metabolite_atp'], color = colors[3], line_width = 2,legend_label = 'atp')
p2_complex.line(timepoints, re['metabolite_pi'], color = colors[6],line_width = 2, legend_label = 'pi')
p2_complex.line(timepoints, re['metabolite_adp'], color = colors[5], line_width = 2,legend_label = 'adp')
p2_complex.legend.location = 'center_right'
p2_complex.legend.click_policy="hide"


p2_complex.line(timepoints, re_atp['metabolite_atp'], legend_label = 'atp only',line_width=2, color = colors[4])
bokeh.io.show(row(p1_complex,p2_complex))

Cool! Moving forward, I attempt to do a parameter search to try to better understand the dynamics of the pathway. I am no longer doing anything particularly different with the software tools shown to us. I have added this code here simply as supplementary information that may be interesting.

## Supplementary
### Parameter Playing

First, I begin by making a slider that allows me to play with different initial conditions and enzyme rates to study the difference between the green atp only curve and the pink (with rheostat) atp curve.<br> 
Here, I define a function (plot_entire) that takes in various rates and initial conditions and outputs plots. It uses the sliders to find the input values for the function.

In [ ]:
k_bf_slider = pn.widgets.FloatSlider(name = 'k_bf', start = 0, end = 100, step = 0.1, value = 20)
k_uf_slider = pn.widgets.FloatSlider(name = 'k_uf', start = 0, end = 100, step = 0.1, value = 20)
k_cat_atp_slider = pn.widgets.FloatSlider(name = 'k_cat_atp', start = 0, end = 100, step = 0.1, value = 1)


enz_init_slider = pn.widgets.FloatSlider(name = 'initial enzyme', start = 0, end = 5, step = 0.01, value = 0.15)
glucose_init_slider = pn.widgets.FloatSlider(name = 'initial glucose', start = 0, end = 200000, step = 1, value = 30)
atp_init_slider = pn.widgets.FloatSlider(name = 'initial atp', start = 0, end = 100, step = 0.01, value = 30)
pi_init_slider = pn.widgets.FloatSlider(name = 'initial pi', start = 0, end = 5000, step = 0.01, value =0)
atpase_init_slider = pn.widgets.FloatSlider(name = 'initial atpase', start = 0, end = 40, step = 0.01, value = 3.5)
t_max_slider = pn.widgets.FloatSlider(name = 't_max', start = 0, end = 1000, step = 1, value = 72)



@pn.depends(k_bf_slider.param.value, k_uf_slider.param.value, k_cat_atp_slider.param.value,
            enz_init_slider.param.value, glucose_init_slider.param.value,
           atp_init_slider.param.value, pi_init_slider.param.value,
           atpase_init_slider.param.value, t_max_slider.param.value)

def plot_entire(k_bf, k_uf, k_cat_atp, enz_init, glucose_init, atp_init, pi_init, atpase_init, t_max):

    k_bf = k_bf 
    k_uf = k_uf 
    
    # Copy Entire Pathway 
    E1_hex = Enzyme(enzyme_name = "hex", substrate = ['glucose'],
                fuel = ['atp'],product = ['g6p'], waste = ['adp'], k_bf = k_bf , k_uf = k_uf)

    E2_pgi = Enzyme(enzyme_name = 'pgi', substrate = ['g6p'], fuel = [],
               product = ['f6p'], waste = [], k_bf = k_bf , k_uf = k_uf)

    E3_pfk = Enzyme(enzyme_name = 'pfk', substrate = ['f6p'], fuel = ['atp'], product = ['f16p'],
               waste = ['adp'], k_bf = k_bf , k_uf = k_uf)

    E4_ald_tpi = Enzyme(enzyme_name ='ald_tpi' , substrate = ['f16p'], fuel = [], product = ['g3p', 'g3p'], 
                waste = [], k_bf = k_bf , k_uf = k_uf )

    E5_gapN = Enzyme(enzyme_name ='gapN' , substrate = ['g3p', 'g3p'], fuel = ['nadp', 'nadp'], product = ['3pg', '3pg'], 
                waste = ['nadph', 'nadph'], k_bf = k_bf , k_uf = k_uf)

    E6_gapM6 = Enzyme(enzyme_name ='gapM6' , substrate = ['g3p', 'g3p'], fuel = ['pi'], product = ['13bpg'], 
                waste = [],k_bf = k_bf , k_uf = k_uf)

    E7_pgk = Enzyme(enzyme_name = 'pgk', substrate = ['13bpg'], fuel = ['adp'], product = ['3pg', '3pg'], 
                waste = ['atp'], k_bf = k_bf , k_uf = k_uf)

    E8_pgm = Enzyme(enzyme_name ='pgm' , substrate = ['3pg', '3pg'], fuel = [], product = ['2pg', '2pg'], 
                waste = [], k_bf = k_bf , k_uf = k_uf)

    E9_eno = Enzyme(enzyme_name ='eno' , substrate = ['2pg', '2pg'], fuel = [], product = ['pep', 'pep'], 
                waste = [],k_bf = k_bf , k_uf = k_uf)

    E10_pyk = Enzyme(enzyme_name = 'pyk', substrate = ['pep', 'pep'], fuel = ['adp', 'adp'], product = ['pyruvate', 'pyruvate'], 
                waste = ['atp', 'atp'], k_bf = k_bf , k_uf = k_uf) # irreversible

    E11_alsS = Enzyme(enzyme_name = 'alsS', substrate = ['pyruvate', 'pyruvate'], fuel = [], product = ['acetolac'], 
                waste = [], k_bf = k_bf , k_uf = k_uf) # irreversible

    E12_IlvC = Enzyme(enzyme_name = 'IlvC', substrate = ['acetolac'], fuel = ['nadph'], product = ['23dih3mebut'], 
                waste = ['nadp'], k_bf = k_bf , k_uf = k_uf)

    E13_IlvD = Enzyme(enzyme_name ='IlvD' , substrate = ['23dih3mebut'], fuel = [], product = ['3me2oxo'], 
                waste = [],k_bf = k_bf , k_uf = k_uf)

    E14_kivD = Enzyme(enzyme_name ='kivD' , substrate = ['3me2oxo'], fuel = [], product = ['isobutanal'], 
                waste = [], k_bf = k_bf , k_uf = k_uf) # irreversible

    E15_yahk = Enzyme(enzyme_name = 'yahk', substrate = ['isobutanal'], fuel = ['nadph'], product = ['isobutanol'],
                      waste = ['nadp'], k_bf = k_bf , k_uf = k_uf)
    
    # Enzyme for ATP Leak
    E16_atpase = Enzyme(enzyme_name = 'atpase', substrate = [], fuel = ['atp'], product = [],
                        waste = ['adp', 'pi'], k_bf = k_bf, k_uf = k_uf, k_cat = k_cat_atp)

    # Define Rheostat + ATP Leak mixture
    myMixture = EnergyTxTl(components = [E1_hex,E2_pgi,E3_pfk, E4_ald_tpi, E5_gapN, E6_gapM6, E7_pgk, E8_pgm, E9_eno, E10_pyk, 
                                        E11_alsS, E12_IlvC, E13_IlvD, E14_kivD, E15_yahk, E16_atpase])
    
    CRN = myMixture.compile_crn()
        
    # Define ATP Leak only mixture   
    myMixture_atp = EnergyTxTl(components = [E16_atpase])
    
    CRN_atp = myMixture_atp.compile_crn()

    
    #CRN.write_sbml_file("CRN.sbml")
    # CRN_atp.write_sbml_file("CRN_atp.sbml")
    
    # Define timepoints and initial conditions
    timepoints = np.linspace(0,t_max,100)
    
    e = enz_init
    atp = atp_init 
    pi = pi_init
    atpase = atpase_init
    x0 = {'molecule_glucose':glucose_init,
          'metabolite_atp': atp,
         'metabolite_nadp':atp,
          'metabolite_pi':pi,
          "enzyme_hex":e,
          'enzyme_pgi':e,
          'enzyme_pfk':e,
          'enzyme_ald_tpi':e,
          'enzyme_gapN':e, 
          'enzyme_gapM6':e,
          'enzyme_pgk':e,
          'enzyme_pgm':e,
          'enzyme_eno':e,
          'enzyme_pyk':e,
          'enzyme_alsS':e,
          'enzyme_IlvC':e,
          'enzyme_IlvD':e,
          'enzyme_kivD':e,
          'enzyme_yahk':e,
          'enzyme_atpase':atpase
    }
    x0_atp = {'metabolite_atp': atp,
               'enzyme_atpase':atpase
    }

    re = CRN.simulate_with_bioscrape(timepoints, initial_condition_dict = x0)
    re_atp = CRN_atp.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_atp)

    colors=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02','#a6761d']

    # First plot glucose and isobutanol
    p3_complex = bokeh.plotting.figure(width = 450, height = 250, 
                             x_axis_label = 'time',
                             y_axis_label = 'concentration (mM)',
                             title = 'Entire Pathway Biocrnpyler with previously used parameters')
    p3_complex.line(timepoints, re['molecule_glucose'], color = colors[0],line_width = 2, legend_label = 'glucose')
    p3_complex.line(timepoints, re['molecule_isobutanol'], color = colors[1], line_width = 2,legend_label = 'isobutanol')
    p3_complex.line(timepoints, re['molecule_f16p'], color = colors[2], line_width = 2,legend_label = 'f16p')
    p3_complex.legend.location = 'center_right'
    p3_complex.legend.click_policy="hide"


    # Plot ATP and ADP, Pi
    p4_complex = bokeh.plotting.figure(width = 450, height = 250,
                               x_axis_label = 'time (hrs)',
                             y_axis_label = 'concentration (mM)')

    p4_complex.line(timepoints, re['metabolite_atp'], color = colors[3], line_width = 1.5,legend_label = 'atp')
    p4_complex.line(timepoints, re_atp['metabolite_atp'], legend_label = 'atp only',line_width=2, color = colors[4])

    p4_complex.legend.location = 'center_right'
    p4_complex.legend.click_policy="hide"
  
    return p4_complex

Now, we will format the sliders and plot the graph.

In [ ]:
widgets = pn.Column(
    pn.Spacer(height=10),
    k_bf_slider, k_uf_slider,
    k_cat_atp_slider,
    enz_init_slider,
    glucose_init_slider,
    atp_init_slider,
    pi_init_slider,
    atpase_init_slider,
    t_max_slider,
    width=150,
)

left_column = pn.Column(
    pn.Row(pn.Spacer(width=30)), plot_entire,
)

# Final layout
pn.Row(left_column, pn.Spacer(width=20), widgets)

By playing with these sliders, we can investigate different ways that the rate constants and initial values affect the curves! We generally try to make sure the green atp_only curve goes to 0 by time 20 since that is what is observed in experimental data.

Now, we define another function (plot_entire2) that takes in parameter values and outputs two plots (one with glucose, isobutanol, f16p, the other with atp, atp only, pi, adp) and the area under the curve of the atp and atp only lines. We decided to use this a sort of metric in attempt to have a quantitative measurement of the dynamics.

In [ ]:
def plot_entire2(k_bf, k_uf, k_cat_atp, enz_init, glucose_init, atp_init, pi_init, nadp_init, atpase_init, t_max, bpg_13 = 0):

    k_bf = k_bf
    k_uf = k_uf

    E1_hex = Enzyme(enzyme_name = "hex", substrate = ['glucose'],
                fuel = ['atp'],product = ['g6p'], waste = ['adp'], k_bf = k_bf , k_uf = k_uf)

    E2_pgi = Enzyme(enzyme_name = 'pgi', substrate = ['g6p'], fuel = [],
               product = ['f6p'], waste = [], k_bf = k_bf , k_uf = k_uf)

    E3_pfk = Enzyme(enzyme_name = 'pfk', substrate = ['f6p'], fuel = ['atp'], product = ['f16p'],
               waste = ['adp'], k_bf = k_bf , k_uf = k_uf)

    E4_ald_tpi = Enzyme(enzyme_name ='ald_tpi' , substrate = ['f16p'], fuel = [], product = ['g3p', 'g3p'], 
                waste = [], k_bf = k_bf , k_uf = k_uf )

    E5_gapN = Enzyme(enzyme_name ='gapN' , substrate = ['g3p', 'g3p'], fuel = ['nadp', 'nadp'], product = ['3pg', '3pg'], 
                waste = ['nadph', 'nadph'], k_bf = k_bf , k_uf = k_uf)

    E6_gapM6 = Enzyme(enzyme_name ='gapM6' , substrate = ['g3p', 'g3p'], fuel = ['pi'], product = ['13bpg'], 
                waste = [],k_bf = k_bf , k_uf = k_uf)

    E7_pgk = Enzyme(enzyme_name = 'pgk', substrate = ['13bpg'], fuel = ['adp'], product = ['3pg', '3pg'], 
                waste = ['atp'], k_bf = k_bf , k_uf = k_uf)

    E8_pgm = Enzyme(enzyme_name ='pgm' , substrate = ['3pg', '3pg'], fuel = [], product = ['2pg', '2pg'], 
                waste = [], k_bf = k_bf , k_uf = k_uf)

    E9_eno = Enzyme(enzyme_name ='eno' , substrate = ['2pg', '2pg'], fuel = [], product = ['pep', 'pep'], 
                waste = [],k_bf = k_bf , k_uf = k_uf)

    E10_pyk = Enzyme(enzyme_name = 'pyk', substrate = ['pep', 'pep'], fuel = ['adp', 'adp'], product = ['pyruvate', 'pyruvate'], 
                waste = ['atp', 'atp'], k_bf = k_bf , k_uf = k_uf) # irreversible

    E11_alsS = Enzyme(enzyme_name = 'alsS', substrate = ['pyruvate', 'pyruvate'], fuel = [], product = ['acetolac'], 
                waste = [], k_bf = k_bf , k_uf = k_uf) # irreversible

    E12_IlvC = Enzyme(enzyme_name = 'IlvC', substrate = ['acetolac'], fuel = ['nadph'], product = ['23dih3mebut'], 
                waste = ['nadp'], k_bf = k_bf , k_uf = k_uf)

    E13_IlvD = Enzyme(enzyme_name ='IlvD' , substrate = ['23dih3mebut'], fuel = [], product = ['3me2oxo'], 
                waste = [],k_bf = k_bf , k_uf = k_uf)

    E14_kivD = Enzyme(enzyme_name ='kivD' , substrate = ['3me2oxo'], fuel = [], product = ['isobutanal'], 
                waste = [], k_bf = k_bf , k_uf = k_uf) # irreversible

    E15_yahk = Enzyme(enzyme_name = 'yahk', substrate = ['isobutanal'], fuel = ['nadph'], product = ['isobutanol'],
                      waste = ['nadp'], k_bf = k_bf , k_uf = k_uf)
    
    E16_atpase = Enzyme(enzyme_name = 'atpase', substrate = [], fuel = ['atp'], product = [],
                        waste = ['adp', 'pi'], k_bf = k_bf, k_uf = k_uf, k_cat = k_cat_atp)


    myMixture = EnergyTxTl(components = [E1_hex,E2_pgi,E3_pfk, E4_ald_tpi, E5_gapN, E6_gapM6, E7_pgk, E8_pgm, E9_eno, E10_pyk, 
                                        E11_alsS, E12_IlvC, E13_IlvD, E14_kivD, E15_yahk, E16_atpase])
    myMixture_atp = EnergyTxTl(components = [E16_atpase])
    
    CRN = myMixture.compile_crn()
    CRN_atp = myMixture_atp.compile_crn()


    
    # CRN.write_sbml_file("CRN.sbml")
    # CRN_atp.write_sbml_file("CRN_atp.sbml")
    
    timepoints = np.linspace(0,t_max,100)
    
    e = enz_init
    atp = atp_init
    pi = pi_init
    atpase = atpase_init
    x0 = {'molecule_glucose':glucose_init,
          'metabolite_atp': atp,
         'metabolite_nadp':nadp_init,
          'metabolite_pi':pi,
          'molecule_13bpg':bpg_13,
          "enzyme_hex":e,
          'enzyme_pgi':e,
          'enzyme_pfk':e,
          'enzyme_ald_tpi':e,
          'enzyme_gapN':e, 
          'enzyme_gapM6':e,
          'enzyme_pgk':e,
          'enzyme_pgm':e,
          'enzyme_eno':e,
          'enzyme_pyk':e,
          'enzyme_alsS':e,
          'enzyme_IlvC':e,
          'enzyme_IlvD':e,
          'enzyme_kivD':e,
          'enzyme_yahk':e,
          'enzyme_atpase':atpase
    }
    x0_atp = {'metabolite_atp': atp,
               'enzyme_atpase':atpase
    }

    re = CRN.simulate_with_bioscrape(timepoints, initial_condition_dict = x0)
    re_atp = CRN_atp.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_atp)

    colors=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02','#a6761d']

    # First plot glucose and isobutanol
    p3_complex = bokeh.plotting.figure(width = 450, height = 250, 
                             x_axis_label = 'time',
                             y_axis_label = 'concentration (mM)',
                             title = 'Entire Pathway Biocrnpyler (Rheostat + ATP Leak)')
    p3_complex.line(timepoints, re['molecule_glucose'], color = colors[0],line_width = 2, legend_label = 'Glucose')
    p3_complex.line(timepoints, re['molecule_isobutanol'], color = colors[1], line_width = 2,legend_label = 'Isobutanol')
    p3_complex.line(timepoints, re['molecule_f16p'], color = colors[2], line_width = 2,legend_label = 'F16P')
    p3_complex.legend.location = 'center_right'
    p3_complex.legend.click_policy="hide"


    # Plot ATP and ADP, Pi
    p4_complex = bokeh.plotting.figure(width = 450, height = 250,
                               x_axis_label = 'time (hrs)',
                             y_axis_label = 'concentration (mM)',
                                      title = 'ATP Compare')


    p4_complex.line(timepoints, re['metabolite_atp'], color = colors[3], line_width = 1.5,legend_label = 'ATP (Rheostat + ATP Leak)')
    p4_complex.line(timepoints, re_atp['metabolite_atp'], legend_label = 'ATP (ATP Leak Only)',line_width=2, color = colors[4])


    #p4_complex.line(timepoints, re['metabolite_pi'], color = 'bisque',line_width = 2, legend_label = 'Pi (Rheostat + ATP Leak)')#colors[6]
    #p4_complex.line(timepoints, re['metabolite_adp'], color = 'lightgray', line_width = 2,legend_label = 'ADP (Rheostat + ATP Leak)')#colors[5]
    p4_complex.legend.location = 'center_right'
    p4_complex.legend.click_policy="hide"
    
    
    # Find area under atp curve
    y = re['metabolite_atp'].values
    dx = timepoints[1] - timepoints[0]
    area1 = trapz(y, dx=dx)
    print('area atp pink',area1)
    
    # Find area under atp only curve
    y2 = re_atp['metabolite_atp'].values
    area2 = trapz(y2, dx=dx)
    print('area atp only green', area2)


    return p3_complex,p4_complex

Now, we can pick parameter values using the sliders in the graph above and plot the results below.

In [ ]:
x1,x2 = plot_entire2(k_bf = 15, k_uf = 15, k_cat_atp = 2.50, enz_init = 0.27, glucose_init = 30,
            atp_init = 30, pi_init = 30, nadp_init = 30, atpase_init = 2., t_max = 72)
bokeh.io.show(row(x1,x2))

In [ ]:
x1,x2 = plot_entire2(k_bf = 15, k_uf = 15, k_cat_atp = 2.50, enz_init = 0.27, glucose_init = 30,
            atp_init = 30, pi_init = 30, nadp_init = 30, atpase_init = 2., t_max = 72)
bokeh.io.show(row(x1,x2))

Now, I performed a parameter search of the different rate constants. Since the code took quite a while to run, I will simply attach the resulting plots here:

<div style="width: 800px; margin: auto">

![non-atp](parameter_search.png)

</div>

The dots are color coded based on the area under the curve of the pink atp line. Using these methods, we continued to play with parameters until we found the most optimal set. Looking forward, we hope to study the flux of metabolites through enzymes, to find parameters that more accurately match experimental data, to attempt to model this system with more accurate TXTL conditions, and simulate other lifetime extension models. <br>
All in all, this class was incredibly helpful and has allowed me to investigate this hypothesis *in silico*. Thank you all for a wonderful term!

In [ ]:
#watermark
%reload_ext watermark
%watermark -v -p numpy,bokeh,jupyterlab,panel,biocrnpyler,bioscrape